В датафреймах (pyspark.sql.DataFrame) заданы продукты, категории и связь
между ними. Одному продукту может соответствовать много категорий,
в одной категории может быть много продуктов. Напишите метод с помощью
PySpark, который вернет все продукты с их категориями (датафрейм с
набором всех пар «Имя продукта – Имя категории»). В результирующем
датафрейме должны также присутствовать продукты, у которых нет категорий.


In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=870f5f55df3fa9305ddd806cfddaeb4877415b8f8ab6a0f84ea4c87d61273ca9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession, DataFrame

In [5]:
spark = SparkSession.builder.appName("pyspark_test").getOrCreate()

In [15]:
categories = spark.createDataFrame([
    (1, "Category 1"),
    (2, "Category 2"),
    (3, "Category 3"),
    (4, "Category 4"),
    (5, "Category 5"),],
    ["id", "category"],
)

In [16]:
products = spark.createDataFrame([
    (1, "Product 1"),
    (2, "Product 2"),
    (3, "Product 3"),
    (4, "Product 4"),
    (5, "Product 5"),
    (6, "Product 6"),
    (7, "Product 7"),
    (8, "Product 8"),
    (9, "Product 9"),
    (10, "Product 10"), ],
    ["id", "product", ]
)

In [17]:
products_vs_categories = spark.createDataFrame([
    (1, 1),
    (2, 3),
    (3, 2),
    (3, 4),
    (6, 4),
    (4, 5),
    (5, 6),
    (6, 7),
    (6, 8),
    (4, 2),
    (1, 8),
    (4, 9),
    (1, 10)],
    ["category_id", "product_id", ]
)

In [18]:
data = (products.join(products_vs_categories,
    products.id == products_vs_categories.product_id, how='left')
    .join(categories,
    products_vs_categories.category_id == categories.id, how='left')
    .select(['product', 'category']).na.fill(value="Not category"))

In [19]:
data.orderBy("product_id", "category_id", ).show(truncate=True)

+----------+------------+
|   product|    category|
+----------+------------+
| Product 1|  Category 1|
| Product 2|  Category 3|
| Product 2|  Category 4|
| Product 3|  Category 2|
| Product 4|  Category 3|
| Product 4|Not category|
| Product 5|  Category 4|
| Product 6|  Category 5|
| Product 7|Not category|
| Product 8|  Category 1|
| Product 8|Not category|
| Product 9|  Category 4|
|Product 10|  Category 1|
+----------+------------+

